In [2]:
sc.addPyFile(<path_to_spark_jobs_zip>)
# sc.addPyFile("s3://mist-data-science-dev/ruchitm/spark_testing/spark_jobs_ruchit_stacktrace.zip")

import logging
import os
import pandas as pd
import random
from datetime import datetime

import pyspark.sql.functions as F
from pyspark.sql.types import *
from common import epoch2datetime
from analytics.utils.time_util import current_epoch_seconds, to_seconds, to_milliseconds
from analytics.jobs.utils import *
from analytics.data_access.utils import parse_search_results

udf_datetime = F.udf(datetime.fromtimestamp, TimestampType())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Data extract and job execute

In [ ]:
date = '2022-06-21'
start_seconds = int((datetime.strptime(date, '%Y-%m-%d')-datetime(1970,1,1)).total_seconds())
debug = True
for i in range(2): # runs for 24 hours duration
    start_time = start_seconds + 3600 * i + 1
    end_time = start_time + 3600 - 1    
    
    job = stats_aggregator_job(start_time, end_time,
                               "es-client-event",
                               spark, "staging", debug_mode=debug)
    job.config['fs_bucket'] = 'mist-data-science-dev'
    job.config['path'] = f"ruchitm/spark_testing/aggregated_stats_{'production' if debug else 'staging'}/"
    if not job.data_source_inst: continue
    data = job.data_source_inst.get_data()
    
    category='mist_universe'
    job.process_category(category, data)

### Check output

In [13]:
date = '2022-06-21'
hour = "*"
prefix = f"s3://mist-data-science-dev/ruchitm/spark_testing/aggregated_stats_production/mist_fast_roaming_failure_stats/"
df_features = spark.read.parquet(prefix+f"dt={date}/hr={hour}")

df_features.printSchema()
df_features_pd = df_features.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- org_id: string (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- failure_count: long (nullable = true)
 |-- stats_agg_time: integer (nullable = true)

In [14]:
df_features_pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                    org_id  ...  stats_agg_time
0     5a578f3c-f094-4d13-a7dd-96f0aaecb5b6  ...      1655769601
1     08e62572-70b1-4405-978e-80220f1bae65  ...      1655769601
2     c1cac1c4-1753-4dde-a065-e17a1c305c2d  ...      1655769601
3     f5451dc6-aa80-4d1c-a49a-dede30b6d878  ...      1655769601
4     a023919c-d30a-4e79-bbd9-135bd68f22c6  ...      1655769601
...                                    ...  ...             ...
3010  a8b71182-9234-4af8-9aec-8fb7896db8aa  ...      1655773201
3011  20b89a2d-ea17-4859-a685-c3125dc32b50  ...      1655773201
3012  bef2f6b3-dffb-44a4-84d9-9f7132e6fa40  ...      1655773201
3013  6ec2e3e5-ea6c-4da3-a04d-c22a0658994f  ...      1655773201
3014  15b211e7-e46e-4b05-8015-dd3fffc11ca3  ...      1655773201

[3015 rows x 4 columns]